# Loan Default Risk Prediction – Real Dataset (GMSC)

This notebook uses the real-world 'Give Me Some Credit' dataset to predict the risk of loan default using machine learning models such as Logistic Regression, Random Forest, and XGBoost.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from xgboost import XGBClassifier
sns.set(style="whitegrid")

In [ ]:
# Load the dataset
df = pd.read_csv("data/gmsc.csv")
df.columns = df.columns.str.strip()
df.drop(columns=[df.columns[0]], inplace=True)
df.rename(columns={"SeriousDlqin2yrs": "default"}, inplace=True)
df['MonthlyIncome'].fillna(df['MonthlyIncome'].median(), inplace=True)
df['NumberOfDependents'].fillna(0, inplace=True)
df.head()

In [ ]:
# Feature engineering
df['debt_ratio_income'] = df['DebtRatio'] * df['MonthlyIncome']
df['loan_to_income'] = df['RevolvingUtilizationOfUnsecuredLines'] * df['MonthlyIncome']
df['log_income'] = np.log1p(df['MonthlyIncome'])
df['log_debt'] = np.log1p(df['debt_ratio_income'])

In [ ]:
# Prepare data for training
X = df.drop("default", axis=1)
y = df["default"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train XGBoost model
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_scaled, y_train)
y_pred_xgb = xgb.predict(X_test_scaled)
y_probs_xgb = xgb.predict_proba(X_test_scaled)[:, 1]
print("Classification Report:\n", classification_report(y_test, y_pred_xgb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))
print("ROC-AUC Score:", roc_auc_score(y_test, y_probs_xgb))

In [ ]:
# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_probs_xgb)
plt.plot(fpr, tpr, label=f'XGBoost AUC = {roc_auc_score(y_test, y_probs_xgb):.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

## ✅ Conclusion
- XGBoost performed well on the GMSC dataset.
- The ROC-AUC score is a reliable indicator of classification performance.
- Further improvements could include SHAP values and hyperparameter tuning.

## 🧠 SHAP Values – Model Explainability
We use SHAP (SHapley Additive exPlanations) to interpret which features influence XGBoost predictions the most.

## 🤖 Model Comparison: Logistic Regression vs Random Forest vs XGBoost
We will now compare three models side-by-side using classification metrics and ROC-AUC.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Train Logistic Regression
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train)
y_probs_lr = lr.predict_proba(X_test_scaled)[:, 1]
y_pred_lr = lr.predict(X_test_scaled)

# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
y_probs_rf = rf.predict_proba(X_test_scaled)[:, 1]
y_pred_rf = rf.predict(X_test_scaled)

In [ ]:
# Compare classification reports
print("Logistic Regression Report:\n", classification_report(y_test, y_pred_lr))
print("Random Forest Report:\n", classification_report(y_test, y_pred_rf))

In [ ]:
# Plot ROC curves
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_probs_lr)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_probs_rf)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_probs_xgb)

plt.figure(figsize=(8, 6))
plt.plot(fpr_lr, tpr_lr, label=f'LogReg AUC = {roc_auc_score(y_test, y_probs_lr):.2f}')
plt.plot(fpr_rf, tpr_rf, label=f'RandomForest AUC = {roc_auc_score(y_test, y_probs_rf):.2f}')
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost AUC = {roc_auc_score(y_test, y_probs_xgb):.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend()
plt.grid()
plt.show()

In [ ]:
import shap
explainer = shap.Explainer(xgb, X_test_scaled)
shap_values = explainer(X_test_scaled)

# Summary plot
shap.summary_plot(shap_values, X_test, max_display=10)

In [3]:
pip install --upgrade numba

/Users/phuongchi/anaconda3/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
     |████████████████████████████████| 2.4MB 5.8MB/s eta 0:00:01
     |████████████████████████████████| 16.9MB 80.4MB/s eta 0:00:01
     |████████████████████████████████| 25.5MB 113.1MB/s eta 0:00:01
ERROR: keras-segmentation 0.3.0 has requirement imgaug==0.2.9, but you'll have imgaug 0.4.0 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4
  Found existing installation: llvmlite 0.29.0
ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install llvmlite

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install numba==0.54.1

/Users/phuongchi/anaconda3/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
     |████████████████████████████████| 2.3MB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 16.0MB 38.0MB/s eta 0:00:01
     |████████████████████████████████| 19.1MB 557kB/s eta 0:00:01
ERROR: keras-segmentation 0.3.0 has requirement imgaug==0.2.9, but you'll have imgaug 0.4.0 which is incompatible.
  Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Found existing installation: llvmlite 0.29.0
ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
Note: you may need to restart the kernel to use updated packages.


## ✅ Final Conclusion
- The XGBoost model achieved strong ROC-AUC performance on a realistic dataset.
- Feature engineering (log income, debt ratio) and stratified sampling helped improve model training.
- SHAP values reveal that credit delinquencies, debt ratios, and income are key drivers of default risk.
- This project demonstrates a practical end-to-end credit risk pipeline using real-world data and explainable ML.